INSTALL
- FAISS - 
- Sent2Vec

In [1]:
import numpy as np
import faiss   

In [2]:
import os
import time
name = "TED-"+ str(time.strftime("%Y%m%d-%H%M"))

!mkdir $name
path = os.getcwd() + "/" + name
os.chdir(path)

!pwd


os.chdir(path)

/home/lw/clc_fairseq/20210113-1520


In [3]:
!wget https://github.com/hoangtrungchinh/clc_data/raw/master/en_vi_iwslt.tar.gz
!mkdir data_bin
!tar -xvf  'en_vi_iwslt.tar.gz' -C 'data_bin'

--2021-01-13 15:20:28--  https://github.com/hoangtrungchinh/clc_data/raw/master/en_vi_iwslt.tar.gz
Resolving github.com (github.com)... 13.250.177.223
Connecting to github.com (github.com)|13.250.177.223|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/en_vi_iwslt.tar.gz [following]
--2021-01-13 15:20:30--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/en_vi_iwslt.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9044100 (8,6M) [application/octet-stream]
Saving to: ‘en_vi_iwslt.tar.gz’

en_vi_iwslt.tar.gz  100%[===================>]   8,62M  4,02MB/s    in 2,1s    

2021-01-13 15:20:33 (4,02 MB/s) - ‘en_vi_iwslt.tar.gz’ saved [9044100/9044100]

IWSLT15.TED.tst2013.en-vi.en
IWSLT15.

In [4]:
# import file
import numpy as np
import glob

en_files = sorted(glob.glob("data_bin/*.en"))
vi_files = sorted(glob.glob("data_bin/*.vi"))
print(en_files)
print(vi_files)

lst_en = []
lst_vi = []

start = time.time()

for file in en_files:
  with open(file) as file_in:
    for line in file_in:
      lst_en.append(line)

for file in vi_files:
  with open(file) as file_in:
    for line in file_in:
      lst_vi.append(line)

print(len(lst_en))
print(len(lst_vi))

['data_bin/IWSLT15.TED.tst2012.en-vi.en', 'data_bin/IWSLT15.TED.tst2013.en-vi.en', 'data_bin/IWSLT15.TED.tst2015.en-vi.en', 'data_bin/train.en']
['data_bin/IWSLT15.TED.tst2012.en-vi.vi', 'data_bin/IWSLT15.TED.tst2013.en-vi.vi', 'data_bin/IWSLT15.TED.tst2015.en-vi.vi', 'data_bin/train.vi']
120956
120956


In [5]:
# SPLIT TRAIN, TEST, VALID
from sklearn.model_selection import train_test_split
en_train, en_test_valid, vi_train, vi_test_valid =  train_test_split(lst_en, lst_vi, test_size=0.2, random_state=123)
en_valid, en_test, vi_valid, vi_test = train_test_split(en_test_valid, vi_test_valid, test_size=0.5, random_state=123)
print(len(en_train))
print(len(vi_train))
print(len(en_valid))
print(len(vi_valid))
print(len(en_test))
print(len(vi_test))
print(en_train[20])
print(vi_train[20])
print(en_valid[-1])
print(vi_valid[-1])
print(en_test[-1])
print(vi_test[-1])

96764
96764
12096
12096
12096
12096
And , in fact , when we did some experiments looking at this , what we found is that 46 percent of the variation in how many friends you have is explained by your genes .

Trong thực tế , khi thực hiện một vài thí nghiệm nghiên cứu vấn đề này , chúng tôi nhận ra 46 % sự khác biệt trong số bạn bè của bạn được giải thích bằng gen của bạn .

This is a chart of what it looked like when it first became popular last summer .

Đây là biểu đồ thể hiện điều đó khi video này lần đầu tiên trở nên phổ biến vào mùa hè trước .

And what will happen in the future ?

Chuyện gì sẽ xảy ra trong tương lai ?



In [6]:
file_variable = [en_train,vi_train,en_valid,vi_valid,en_test,vi_test]
file_names = ["en_train","vi_train","en_valid","vi_valid","en_test","vi_test"]

for i in range(len(file_names)):
  with open(file_names[i], 'w') as f:
    for item in file_variable[i]:
      f.write(item)
!ls -al

total 36844
drwxrwxr-x 3 lw lw     4096 Thg 1  13 15:20 .
drwxrwxr-x 7 lw lw     4096 Thg 1  13 15:20 ..
drwxrwxr-x 2 lw lw     4096 Thg 1  13 15:20 data_bin
-rw-rw-r-- 1 lw lw  1260181 Thg 1  13 15:20 en_test
-rw-rw-r-- 1 lw lw 10011509 Thg 1  13 15:20 en_train
-rw-rw-r-- 1 lw lw  1249342 Thg 1  13 15:20 en_valid
-rw-rw-r-- 1 lw lw  9044100 Thg 1  13 15:20 en_vi_iwslt.tar.gz
-rw-rw-r-- 1 lw lw  1624771 Thg 1  13 15:20 vi_test
-rw-rw-r-- 1 lw lw 12896000 Thg 1  13 15:20 vi_train
-rw-rw-r-- 1 lw lw  1609835 Thg 1  13 15:20 vi_valid


In [7]:
start = time.time()
import sent2vec
model = sent2vec.Sent2vecModel()
model.load_model('../wiki_unigrams.bin')

print('=== Ending, Total time (second): ', time.time() - start)

=== Ending, Total time (second):  6.503113031387329


In [8]:
srcfile = "en_train"
tarfile = "vi_train"
# lst_srcfile = open(srcfile, "r").readlines()
# lst_tarfile = open(tarfile, "r").readlines()
lst_srcfile = en_train
lst_tarfile = vi_train

vec_lst_srcfile = model.embed_sentences(lst_srcfile)

Test Cosin similarity

In [9]:
import faiss

start = time.time()
vec_lst_srcfile = model.embed_sentences(lst_srcfile)
vec_lst_srcfile2 = model.embed_sentences(lst_srcfile)
x = vec_lst_srcfile
q = vec_lst_srcfile2
index = faiss.IndexFlatL2(600)
# index = faiss.index_factory(3, "Flat", faiss.METRIC_INNER_PRODUCT)
index.ntotal
faiss.normalize_L2(x)
index.add(x)
faiss.normalize_L2(q)
k = 6
distance, index = index.search(q, k)

print('=== Ending, Total time (second): ', time.time() - start)
print(len(distance))
print(len(index))

print(index)
print(distance)

=== Ending, Total time (second):  76.73803758621216
=== Ending, Total time (second):  76.73821020126343
[[    0 35305 41676 43042  4311  4864]
 [    1 69317 55307 20678 90041  5955]
 [    2 32305 19196  3145 74668 49940]
 ...
 [96761 23129  4275 71607 12392 67164]
 [96762  1864 77275 74179 54036 28801]
 [96763 85347 21184 44165 24605 52166]]
[[0.0000000e+00 6.6998875e-01 7.3994982e-01 7.4247205e-01 7.4674928e-01
  7.5621331e-01]
 [0.0000000e+00 6.7452312e-01 6.7536390e-01 6.8550754e-01 6.9720864e-01
  7.0347011e-01]
 [0.0000000e+00 5.8817363e-01 6.4495087e-01 6.6965878e-01 6.7031384e-01
  6.7404485e-01]
 ...
 [0.0000000e+00 7.9059279e-01 7.9934967e-01 8.1222677e-01 8.2138360e-01
  8.2454550e-01]
 [0.0000000e+00 9.4278812e-01 1.0312700e+00 1.0327064e+00 1.0494604e+00
  1.0516963e+00]
 [4.7683716e-07 6.7013419e-01 7.6927340e-01 7.8463757e-01 8.8304937e-01
  9.2391801e-01]]


In [11]:
i = 6
j = k-1
print(index[i][j])
print("==================================")
print(distance[i][j])
a = vec_lst_srcfile[i]
# print(a)
b = vec_lst_srcfile[index[i][j]]
# TEST
dist_squared = np.sum(np.square(a - b))
print(dist_squared)

31083
0.70538425
0.7053839


In [12]:
start = time.time()
saperate = " || "
threshold = 0.8
src_label = "S"
emb_label = "E"

out_file_write = open("en_train_EM", 'w')
out_file_write_score = open("en_train_EM_score", 'w')
out_file_write_factor = open("en_train_EM_factor", 'w')

for i in range(len(lst_srcfile)):
    score = distance[i][1]
    content = ""
    content_label_arr = []
    if score >= threshold:
        best_simi_index = index[i][1]
        content = lst_srcfile[i].strip() + saperate + lst_tarfile[best_simi_index].strip()
        
        content_label = [src_label] * len(lst_srcfile[i].split())
        content_label.append(emb_label)
        content_label = content_label + [emb_label] * len(lst_tarfile[best_simi_index].split())
        
        content_label_arr = " ".join(content_label)
        
        
   
    else:
        content = lst_srcfile[i].strip()
        content_label = [src_label] * len(lst_srcfile[i].split())
        content_label_arr = " ".join(content_label)
        
        
    if i<10:
        print(content)
        print(len(content.split()))  
        print(len(content_label))
        print("======================")
        

    out_file_write.writelines(content+ "\n")    
    out_file_write_factor.writelines(content_label_arr + "\n")
    out_file_write_score.writelines(str(score) + "\n")
        

out_file_write.close()

done = time.time()
elapsed = done - start
print('=== Ending, Total time (second): ', elapsed)

It &apos;s like the guy is sitting next to you , whispering who knows what into your ear .
19
19
They go bad very quickly if you don &apos;t , and if you don &apos;t have stable power grid , this doesn &apos;t happen , so kids die .
29
29
Now let me ask you this question : This past week at TED , how many of you , when you saw vulnerability up here , thought it was pure courage ?
32
32
My archive : 175 films , my 16-millimeter negative , all my books , my dad &apos;s books , my photographs .
22
22
Now , a few weeks before our vote , Lake Superior State University issues its list of banished words for the year . || Cứ đến tháng Một , chúng tôi đến cuộc họp thường niên Hội Phương ngữ Mỹ trong nhiều việc làm ở đó có bầu chọn Từ của năm .
53
53
It &apos;s you and I who make our fates -- we open up or close the gates on the road ahead or the road behind . &quot; || &quot; Và chúng tôi muốn xúc tiến và kết thúc vụ này .
41
41
Now this is several years ago . Adjusted for inflation , it &apos;